# Retrieval Augmented Generation

In [1]:
pip install -q langchain-community langchain_aws 


[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain_aws import ChatBedrock

llm = ChatBedrock(
    model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    model_kwargs=dict(temperature=0),
    region="us-east-1"
)


In [3]:
ai_msg = llm.invoke("Which VLANs are reverved in my network?")
print(ai_msg.content)


# Reserved VLANs

In standard Ethernet networks, certain VLANs are typically reserved:

- **VLAN 1**: Default VLAN (often used for management)
- **VLANs 1002-1005**: Reserved for legacy protocols:
  - 1002: FDDI
  - 1003: Token Ring
  - 1004: FDDI Net
  - 1005: Token Ring Net

On Cisco equipment, VLANs 1001-1005 are automatically created and cannot be deleted.

Without specific information about your particular network configuration, I can only provide these standard reservations. Your organization may have additional reserved VLANs based on your network design and policies.


## Ingestion Pipeline (Add data to Vector DB)
### Load

In [5]:
from langchain.document_loaders import TextLoader

loader = TextLoader("acme_policies.md")
documents = loader.load()


### Split

In [7]:
from langchain_text_splitters import MarkdownHeaderTextSplitter

headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]
md_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)

In [13]:
md_splits = md_splitter.split_text(documents[0].page_content)
md_splits[5].metadata

{'Header 1': 'Policies', 'Header 2': 'Quality of Service (QoS)'}

In [14]:
md_splits[5].page_content

'1. All edge interfaces must implement QoS policies to prioritize voice and critical applications.  \n```\npolicy-map EDGE_POLICY\nclass VOICE\npriority 500\nclass CRITICAL_APP\nbandwidth percent 30\n```  \n2. Default traffic should be marked as Best Effort (BE).'

In [19]:
len(md_splits)

9

### Embedd

In [18]:
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_aws import BedrockEmbeddings

embeddings = BedrockEmbeddings(
    model_id="amazon.titan-embed-text-v2:0",
    region_name='us-east-1'
)

vector_store = InMemoryVectorStore(embeddings)
vector_store.add_documents(documents=md_splits)

['df796a1d-6ede-43a1-9e09-b9e46026272c',
 '6c7074df-b71e-4747-a8f1-5edb71f8843c',
 '88041a4d-27f9-4af0-9b38-5a6ffa1c9b3a',
 '157f2e2a-1802-4d8a-87ac-ab51a2de7e55',
 'fe5a6165-3204-44b7-b610-eb640fd20914',
 'd30a7299-f712-4674-b158-9352c204adbe',
 '04d0e48a-f7cd-4bda-b9f4-e2cc2611823d',
 'abf62948-72b1-4904-a7cd-ef269ba1a127',
 'feb55661-afbf-4397-80eb-fb333ad9fed8']

In [27]:
for index, (id, doc) in enumerate(vector_store.store.items()):
    if index < 2:
        print(doc['text'])
        print(f"{id}: {doc['vector'][0:5]} ({len(doc['vector'])})")
        print('----------------------------------------------------------------------------------------------------')

The following document states the configuration policies of the devices of the ACME company.
df796a1d-6ede-43a1-9e09-b9e46026272c: [-0.030192214995622635, 0.060329172760248184, -0.00401478772982955, -0.007488475181162357, 0.0706123635172844] (1024)
----------------------------------------------------------------------------------------------------
1. Routable addresses used on physical Interfaces (E.g: Ethernet 1/2), Port-Chanels (port-channel1) or SVI Interfaces must be taken from the CIDR 10.1.0.0/16  
Example:
```
interface vlan 123
no switchport
ip address 10.1.1.254/24
```  
2. IP Addresses assigned to Router IDs (RID) of routing protocols such as OSPF, EIGRP or BGP must be taken from the CIDR 172.20.0.0/16  
Example:
```
router ospf OSPF1
router-id  172.20.1.5
```
6c7074df-b71e-4747-a8f1-5edb71f8843c: [-0.06210282817482948, 0.028775574639439583, 0.0035110258031636477, 0.07173123955726624, 0.04515642300248146] (1024)
----------------------------------------------------------------

### Similarity Search

In [28]:
query = "approved security mechanisms"
results = vector_store.similarity_search(query, k=2)
for result in results:
    print(result.page_content)

1. The only valid hashing protocol for payload integrity is MD5. This applies for SNMP configurations and authenticated BGP peerings.  
```
snmp-server user MYUSER MYGROUP v3 auth sha <password> priv aes 256 <password>
```  
2. The only valid authentication protocol for payload encryption is AES 256.
The following document states the configuration policies of the devices of the ACME company.


### RAG Pipeline (Retrieve context from Vector DB)

In [39]:
from langchain import hub
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import PromptTemplate

retrieval_qa_chat_prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""
You are a network engineer assistant. Use the following documentation to answer the question.'
If the answer is not in the documentation, say you don’t know.

Documentation:
{context}

Question: {question}

Answer:
"""
)

In [40]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)
    
rag_chain = (
    {'context': vector_store.as_retriever() | format_docs, 'question' : RunnablePassthrough()} |
    retrieval_qa_chat_prompt |
    llm
)
result = rag_chain.invoke("Which VLANs are reverved in the ACME network?")
print(result.content)

Based on the documentation provided, the following VLANs are reserved in the ACME network:

- Management VLAN: 10
- Voice VLAN: 20
- Guest VLAN: 30

These are specifically mentioned in section 2 of the documentation as "Reserved VLANs for management and infrastructure."
